<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9回目の提出さ

ここまでのtry
*   Light GBM、特徴量12個くらい。MinMaxScalerで正規化
*   基準を0.1以上をOKとしたらクリアになった。
*   これは完全に過学習を起こしている。
*　 crossvalidationがちゃんと働いてないな。
*　 cataegorical featuresという形でもうできることがわかったぞ


---


今後の課題
*   optunaを入れて調整しよう
*   過学習の判断をできるようにしよう
*   Minmaxとかの正規化ってcategoricaldataにもやっていいの？
*   
*   


In [1]:
#importまとめ。pandas-profilingは少々導入が面倒
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file
%matplotlib inline

warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-y39nhgmu
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-y39nhgmu
     |████████████████████████████████| 71kB 298kB/s 
     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 604kB 4.6MB/s 
     |████████████████████████████████| 3.1MB 7.6MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 296kB 36.2MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=4c395ad3d1884760306598af12d2e370471a280dc9e6a50450181ced89649db7
  Stored in directory: /tmp/pip-ephem-wheel-cache-7dfqw0d7/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=7de2cf4375b0d81debd0bd1228616fb11ce1745418183bfe07c6235b800418a2


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data


In [4]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

Saving submit_sample.csv to submit_sample.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [36]:
!ls

sample_data		       submit_sample.csv  train.csv
submission_lightgbm_kfold.csv  test.csv


In [66]:
#上手く列指定ができなかったので、先頭にheaderをつける
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
#test,train,その足し合わせdataの三種を作る
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["job"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["marital"].replace(['married','single','divorced'],[2,1,0],inplace=True)
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,2,1,0],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,2,1,0],inplace=True)
data["job"] = data["job"].astype(np.int64)

#全体のうち、消したいモノだけ選ぶ
all_columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome']
remain_columns = list(set(all_columns)-set(delete_columns))
remain_columns = ['age','job','marital','education','housing','loan','contact','campaign','previous','poutcome']
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]


**------------------cross validation--------------------**


**------light GBM-------**

In [67]:
#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y_train = train['y']
X_train = train.drop('y', axis = 1)
X_test = test.drop('y', axis = 1)

# MinmaxScalarを使って正規化。
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
# 訓練用のデータを正規化する。fit_transformはfit＆transformの略で、情報取得＆正規化の意味
X_train_norm = mmsc.fit_transform(X_train)
# 訓練用データを基準にテストデータも正規化
X_test_norm = mmsc.transform(X_test)
X_train=X_train_norm
X_test=X_test_norm
#正規化するとarrayになっていたので、DataFrameに戻す
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
#Dataflameにしたらheaderが抜けていたので付け直す
X_train.columns=remain_columns
X_test.columns=remain_columns
#この辺のheader周りの操作は無駄なのでできたら修正したい

In [68]:
#欲しいデータが見えているか確認
data

,age,job,marital,education,housing,loan,contact,campaign,previous,poutcome,y
0,31,4,2,3,1,0,2,3,0,0,0.0
1,29,7,1,2,0,0,2,2,0,1,1.0
2,35,1,2,2,1,0,2,1,0,2,0.0
3,31,2,2,3,1,1,0,2,0,2,0.0
4,48,10,2,1,1,0,1,1,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...
18045,49,6,2,2,1,0,2,2,0,2,NaN
18046,34,0,2,3,1,0,2,1,0,1,NaN
18047,34,3,1,3,1,0,0,2,0,1,NaN
18048,31,2,1,3,1,0,0,2,0,1,NaN


In [69]:
remain_columns

['age',
 'job',
 'marital',
 'education',
 'housing',
 'loan',
 'contact',
 'campaign',
 'previous',
 'poutcome']

In [70]:
X_train

,age,job,marital,education,housing,loan,contact,campaign,previous,poutcome
0,0.132353,0.363636,1.0,1.000000,1.0,0.0,1.0,0.50,0.0,0.000000
1,0.102941,0.636364,0.5,0.666667,0.0,0.0,1.0,0.25,0.0,0.333333
2,0.191176,0.090909,1.0,0.666667,1.0,0.0,1.0,0.00,0.0,0.666667
3,0.132353,0.181818,1.0,1.000000,1.0,1.0,0.0,0.25,0.0,0.666667
4,0.382353,0.909091,1.0,0.333333,1.0,0.0,0.5,0.00,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...
27095,0.220588,0.000000,1.0,1.000000,1.0,0.0,1.0,0.75,0.0,0.333333
27096,0.191176,0.363636,1.0,1.000000,1.0,0.0,1.0,0.00,0.0,0.333333
27097,0.191176,0.363636,1.0,0.000000,0.0,0.0,1.0,0.25,0.0,0.333333
27098,0.117647,0.272727,0.5,1.000000,1.0,0.0,1.0,0.00,0.0,0.333333


In [71]:
#LightGBMに入れる。
from sklearn.model_selection import KFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

#categorical dataの入れ忘れに注意
categorical_features = ["job","marital","education","housing","loan","contact","poutcome"]

params = {
    'objective': 'binary',
    'max_bin': 400,
    'learning_rate': 0.01,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)
    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)
    
    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.266131	valid_1's binary_logloss: 0.244256
[20]	training's binary_logloss: 0.257993	valid_1's binary_logloss: 0.2385
[30]	training's binary_logloss: 0.252018	valid_1's binary_logloss: 0.234323
[40]	training's binary_logloss: 0.247314	valid_1's binary_logloss: 0.231178
[50]	training's binary_logloss: 0.243505	valid_1's binary_logloss: 0.228754
[60]	training's binary_logloss: 0.240386	valid_1's binary_logloss: 0.226694
[70]	training's binary_logloss: 0.237785	valid_1's binary_logloss: 0.225113
[80]	training's binary_logloss: 0.235624	valid_1's binary_logloss: 0.22377
[90]	training's binary_logloss: 0.233787	valid_1's binary_logloss: 0.222707
[100]	training's binary_logloss: 0.23221	valid_1's binary_logloss: 0.221872
[110]	training's binary_logloss: 0.230794	valid_1's binary_logloss: 0.221172
[120]	training's binary_logloss: 0.229542	valid_1's binary_logloss: 0.220587
[130]	training's binary_log

In [72]:
#pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.21807171488284322, 0.22905937126152875, 0.23228758032599517, 0.2282601283603254, 0.2323082736079079]
0.22799741368772009


In [77]:
from sklearn.metrics import accuracy_score
percentage=0.5

y_pred_oof = (oof_train > percentage).astype(int)
accuracy_score(y_train, y_pred_oof)

0.9291512915129151

In [78]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > percentage).astype(int)


sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(226, 18050)

In [79]:
sub

,A,B
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
18045,18045,0
18046,18046,0
18047,18047,0
18048,18048,0


-----ここまでcross validation-------

In [80]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>